# 5월 26일

# 두번째 프로젝트

## K - Means 알고리즘






- csv 파일(데이터)을 넣어주면
- 데이터를 그룹핑( = clustering) 해 주는 앱을 만들어보자

#### 참고 ) 용어 구분
- regression 회귀 : 숫자 예측.

  - 관찰된 연속형 변수들에 대해 두 변수 사이의 모형을 구한 뒤 적합도를 측정해 내는 분석 방법.
  - 정답이 있어야 함.

- classification 분류 : 
정해져있는것이 있는데 이게 그 중 어디에 들어갈까?


- clustering 군집 :

  - (예시) 주문 내역 보고 주문 내역이 비슷한 사람들끼리 묶어보면 어떤 특징을 알 수 있지 않을까?

  - (예시) 불량이 많이 나오는데 어디서 불량이 나오는지 그룹핑을 한번 해볼까?

#### 참고) 용어 정의

regression, classification : 슈퍼바이즈드 러닝 

clustering : 언슈퍼바이즈드 러닝


==>> 슈퍼바이즈드 러닝이란?

: 감독된 기계 학습 알고리즘은 라벨로 표시된 예를 사용하여 미래 사건을 예측하는 새로운 데이터에 과거 학습한 내용을 적용


==>> 언슈퍼바이즈드 러닝이란?

: 훈련하는 데 사용되는 정보가 분류되지 않거나 라벨이 붙어 있지 않을 때, 감독되지 않는 기계 학습 알고리즘이 사용

In [ ]:
### 다음 데이터로 집값 예측하는 방법 (regression)

# 방 수, 화장실 수, 평수, 층수, 지역
# 3, 2, 70, 17, 청라
# 2, 1, 20, 3, 계양

# -->> 못함. 왜? 집값이 없음!!

#### 데이터 분석

##### 데이터 불러오기

In [3]:
import pandas as pd
import matplotlib.pyplot as plt




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# pd.read_csv('/content/drive/MyDrive/Colab Notebooks/yh-k4-main2-원본용/data/Mall_Customers.csv')

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/yh-k4-main2-원본용/data/Mall_Customers.csv')

##### 데이터 정보 확인

In [6]:
df

,CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40
...,...,...,...,...,...
195,196,Female,35,120,79
196,197,Female,45,126,28
197,198,Male,32,126,74
198,199,Male,32,137,18


In [7]:
df.describe()

,CustomerID,Age,Annual Income (k$),Spending Score (1-100)
count,200.000000,200.000000,200.000000,200.000000
mean,100.500000,38.850000,60.560000,50.200000
std,57.879185,13.969007,26.264721,25.823522
min,1.000000,18.000000,15.000000,1.000000
25%,50.750000,28.750000,41.500000,34.750000
50%,100.500000,36.000000,61.500000,50.000000
75%,150.250000,49.000000,78.000000,73.000000
max,200.000000,70.000000,137.000000,99.000000


In [8]:
df['Annual Income (k$)'].max()

137

In [9]:
df['Annual Income (k$)'] == df['Annual Income (k$)'].max()

0      False
1      False
2      False
3      False
4      False
       ...  
195    False
196    False
197    False
198     True
199     True
Name: Annual Income (k$), Length: 200, dtype: bool

In [11]:

# 가장 수입이 많은 사람

df.loc[   df['Annual Income (k$)'] == df['Annual Income (k$)'].max() ,   ]

,CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
198,199,Male,32,137,18
199,200,Male,30,137,83


In [12]:

# 가장 소비 점수가 높은 사람

df.loc[   df['Spending Score (1-100)'] == df['Spending Score (1-100)'].max() ,   ]

,CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
11,12,Female,35,19,99


In [13]:

# 가장 나이가 많은 사람

df.loc[   df['Age'] == df['Age'].max() ,   ]

,CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
60,61,Male,70,46,56
70,71,Male,70,49,55


In [14]:

# 비어있는 데이터 확인

df.isna().sum()

# 결측치가 있다면 df = df.dropna( inplace = True ) 써서 비어있는데이터 없애주기 
 # (inplace = True 쓰면 메모리에 변경해서 저장해라 라는 의미)

CustomerID                0
Genre                     0
Age                       0
Annual Income (k$)        0
Spending Score (1-100)    0
dtype: int64

##### 데이터 학습하기

In [15]:
df.head(2)

,CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81


In [ ]:
#### 무엇을 y로 해야 할까?

# 목적 : 비슷한 사람들끼리 묶기 위해
# 묶인 그룹을 예측해야함.
# 컴퓨터니까 그룹정보는 0,1,2,3... 이런식으로 숫자로 나와야함.

# 여기서 -->> y는 없고 X만 세팅 하면 됨.

In [17]:
#  행은 다 써야함, 행은 1번인덱스부터 끝까지

X = df.iloc[  :  ,  1: ]

# df.iloc[  :  ,  1: ].values 붙이면 넘파이로 학습함

In [18]:
X

,Genre,Age,Annual Income (k$),Spending Score (1-100)
0,Male,19,15,39
1,Male,21,15,81
2,Female,20,16,6
3,Female,23,16,77
4,Female,31,17,40
...,...,...,...,...
195,Female,35,120,79
196,Female,45,126,28
197,Male,32,126,74
198,Male,32,137,18


In [22]:

# 젠더는 숫자가 아니기때문에 숫자로 바꿔줘야함. ===>> '인코딩' 한다고 함.
# 숫자로 바꿔주는 인코딩 : 레이블 인코딩, 원 핫 인코딩


# 데이터 종류가 몇개나 있느냐
df['Genre'].nunique()

2

In [23]:

from sklearn.preprocessing import LabelEncoder, OneHotEncoder


In [24]:

# 라이브를 가져오면 항상 변수에 저장해둬야 함

label_encoder = LabelEncoder()

In [26]:

# 젠더 컬럼에 데이터를 숫자로 바꿔달라
# 바뀌는 순서는 알파벳순


# 바꾼걸 다시 컬럼으로 집어 넣어야 함.

X['Genre'] = label_encoder.fit_transform(  X['Genre']  )

In [27]:
X

,Genre,Age,Annual Income (k$),Spending Score (1-100)
0,1,19,15,39
1,1,21,15,81
2,0,20,16,6
3,0,23,16,77
4,0,31,17,40
...,...,...,...,...
195,0,35,120,79
196,0,45,126,28
197,1,32,126,74
198,1,32,137,18


In [28]:

# 각 컬럼별 범위 확인
# 각 컬럼별 ㅁ범위를 통일시켜줘야 함

X.describe()


,Genre,Age,Annual Income (k$),Spending Score (1-100)
count,200.000000,200.000000,200.000000,200.000000
mean,0.440000,38.850000,60.560000,50.200000
std,0.497633,13.969007,26.264721,25.823522
min,0.000000,18.000000,15.000000,1.000000
25%,0.000000,28.750000,41.500000,34.750000
50%,0.000000,36.000000,61.500000,50.000000
75%,1.000000,49.000000,78.000000,73.000000
max,1.000000,70.000000,137.000000,99.000000


##### 피쳐 스케일링

In [29]:

from sklearn.preprocessing import MinMaxScaler, StandardScaler


In [30]:
scaler = MinMaxScaler()

In [31]:

scaler.fit_transform(  X  )


array([[1.        , 0.01923077, 0.        , 0.3877551 ],
       [1.        , 0.05769231, 0.        , 0.81632653],
       [0.        , 0.03846154, 0.00819672, 0.05102041],
       [0.        , 0.09615385, 0.00819672, 0.7755102 ],
       [0.        , 0.25      , 0.01639344, 0.39795918],
       [0.        , 0.07692308, 0.01639344, 0.76530612],
       [0.        , 0.32692308, 0.02459016, 0.05102041],
       [0.        , 0.09615385, 0.02459016, 0.94897959],
       [1.        , 0.88461538, 0.03278689, 0.02040816],
       [0.        , 0.23076923, 0.03278689, 0.7244898 ],
       [1.        , 0.94230769, 0.03278689, 0.13265306],
       [0.        , 0.32692308, 0.03278689, 1.        ],
       [0.        , 0.76923077, 0.04098361, 0.14285714],
       [0.        , 0.11538462, 0.04098361, 0.7755102 ],
       [1.        , 0.36538462, 0.04098361, 0.12244898],
       [1.        , 0.07692308, 0.04098361, 0.79591837],
       [0.        , 0.32692308, 0.04918033, 0.34693878],
       [1.        , 0.03846154,

In [32]:
X_scaled = scaler.fit_transform(  X  )

In [33]:
X_scaled

array([[1.        , 0.01923077, 0.        , 0.3877551 ],
       [1.        , 0.05769231, 0.        , 0.81632653],
       [0.        , 0.03846154, 0.00819672, 0.05102041],
       [0.        , 0.09615385, 0.00819672, 0.7755102 ],
       [0.        , 0.25      , 0.01639344, 0.39795918],
       [0.        , 0.07692308, 0.01639344, 0.76530612],
       [0.        , 0.32692308, 0.02459016, 0.05102041],
       [0.        , 0.09615385, 0.02459016, 0.94897959],
       [1.        , 0.88461538, 0.03278689, 0.02040816],
       [0.        , 0.23076923, 0.03278689, 0.7244898 ],
       [1.        , 0.94230769, 0.03278689, 0.13265306],
       [0.        , 0.32692308, 0.03278689, 1.        ],
       [0.        , 0.76923077, 0.04098361, 0.14285714],
       [0.        , 0.11538462, 0.04098361, 0.7755102 ],
       [1.        , 0.36538462, 0.04098361, 0.12244898],
       [1.        , 0.07692308, 0.04098361, 0.79591837],
       [0.        , 0.32692308, 0.04918033, 0.34693878],
       [1.        , 0.03846154,